In [ ]:
%matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from sklearn.model_selection import ParameterGrid
from torchvision.datasets import ImageFolder
TRAIN_PATH = "train"
TEST_PATH ="test"



In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive


In [ ]:
import zipfile


zip_ref = zipfile.ZipFile('/content/drive/Shareddrives/Datasets SEFAI/fer2013.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:

def mean_std_calc(loader):
  cnt = 0
  fst_moment = torch.empty(3)
  snd_moment = torch.empty(3)

  for images, _ in loader:
      b, c, h, w = images.shape
      nb_pixels = b * h * w
      sum_ = torch.sum(images, dim=[0, 2, 3])
      sum_of_square = torch.sum(images ** 2,
                                dim=[0, 2, 3])
      fst_moment = (cnt * fst_moment + sum_) / (
                    cnt + nb_pixels)
      snd_moment = (cnt * snd_moment + sum_of_square) / (
                          cnt + nb_pixels)
      cnt += nb_pixels

  mean, std = fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)
  return mean, std


In [ ]:
train_dir = "train" # Directory containing the training data
test_dir = "test"  # Directory containing the validation data
#Normalizzazione train loader


train_dataset = ImageFolder(train_dir, transform=transforms.ToTensor())
test_dataset = ImageFolder(test_dir, transform=transforms.ToTensor())

train_loader =  DataLoader(train_dataset, batch_size=64, shuffle=False)
mean, std = mean_std_calc(train_loader)

# Define the transforms
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=mean, std=std),

])

test_loader =  DataLoader(test_dataset, batch_size=64, shuffle=False)
mean, std = mean_std_calc(test_loader)

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=mean, std=std)
])

train_dataset = ImageFolder(train_dir, transform=train_transform)
test_dataset = ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, dim=1)
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc, n_correct;

In [ ]:

class Vgg_face_dag(nn.Module):

    def __init__(self):
        super(Vgg_face_dag, self).__init__()
        self.meta = {'mean': [129.186279296875, 104.76238250732422, 93.59396362304688],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.fc6 = nn.Linear(in_features=25088, out_features=4096, bias=True)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(p=0.5)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(p=0.5)
        self.fc8 = nn.Linear(in_features=4096, out_features=2622, bias=True)
        self.relu8 = nn.ReLU(inplace=True)
        self.dropout8 = nn.Dropout(p=0.5)
        self.fc9 = nn.Linear(in_features=2622, out_features=7, bias=True)

    def forward(self, x0):
        x1 = self.conv1_1(x0)
        x2 = self.relu1_1(x1)
        x3 = self.conv1_2(x2)
        x4 = self.relu1_2(x3)
        x5 = self.pool1(x4)
        x6 = self.conv2_1(x5)
        x7 = self.relu2_1(x6)
        x8 = self.conv2_2(x7)
        x9 = self.relu2_2(x8)
        x10 = self.pool2(x9)
        x11 = self.conv3_1(x10)
        x12 = self.relu3_1(x11)
        x13 = self.conv3_2(x12)
        x14 = self.relu3_2(x13)
        x15 = self.conv3_3(x14)
        x16 = self.relu3_3(x15)
        x17 = self.pool3(x16)
        x18 = self.conv4_1(x17)
        x19 = self.relu4_1(x18)
        x20 = self.conv4_2(x19)
        x21 = self.relu4_2(x20)
        x22 = self.conv4_3(x21)
        x23 = self.relu4_3(x22)
        x24 = self.pool4(x23)
        x25 = self.conv5_1(x24)
        x26 = self.relu5_1(x25)
        x27 = self.conv5_2(x26)
        x28 = self.relu5_2(x27)
        x29 = self.conv5_3(x28)
        x30 = self.relu5_3(x29)
        x31_preflatten = self.pool5(x30)
        x31 = x31_preflatten.view(x31_preflatten.size(0), -1)
        x32 = self.fc6(x31)
        x33 = self.relu6(x32)
        x34 = self.dropout6(x33)
        x35 = self.fc7(x34)
        x36 = self.relu7(x35)
        x37 = self.dropout7(x36)
        x38 = self.fc8(x37)
        x39 = self.relu8(x38)
        x40 = self.dropout8(x39)
        x41 = self.fc9(x40)
        return x41

def vgg_face_dag(weights_path=None, **kwargs):
    """
    load imported model instance

    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Vgg_face_dag()
    pretrained_dict = torch.load(weights_path)
    model_dict = model.state_dict()

    # I pesi caricati vengono filtrati per assicurarsi che vengano caricati solo quelli di layer/neuroni effettivamente presenti
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}

    # Carica i pesi preaddestrati nel modello personalizzato
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    # Freeze dei pesi già presenti (tranne ultimo layer)
    #for name, param in model.named_parameters():
     #   if name not in ['fc8.weight', 'fc8.bias', 'relu8.weight', 'relu8.bias', 'dropout8.weight', 'dropout8.bias', 'fc9.weight', 'fc9.bias']:
      #      param.requires_grad = False
      #Commentato perché funziona peggio rispetto a che tutti i pesi subiscano il grad. desc.

    return model


In [ ]:
model = vgg_face_dag(weights_path='/content/drive/Shareddrives/Datasets SEFAI/vgg-pretrained/robots.ox.ac.uk_~albanie_models_pytorch-mcn_vgg_face_dag.pth').to(device)



In [ ]:

acc_list_train=[]
acc_list_test=[]


n_total_steps = len(train_loader)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum =0.9)

patience = 3



best_loss = 100
counter=0
stop=False
num_epochs = 100
for epoch in range(num_epochs):
        model.train()
        print(counter)
        if stop:
          print(stop)
          break
        running_loss = 0.0
        running_acc = 0
        seen = 0
        for images, labels in train_loader:

          images = images.to(device)
          labels = labels.to(device)


          outputs = model(images)

          loss = criterion(outputs, labels)

          _, acc = accuracy(outputs, labels)
          seen +=labels.shape[0]

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
          running_acc += acc

        print (f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Acc: {running_acc/seen:.4f}')
        acc_list_train.append(running_acc/len(train_loader))
        model.eval()

        tot_corrette = 0
        tot_eseguite = 0
        running_test_loss = 0
        val_loss = 0

        with torch.no_grad():

          for images, labels in test_loader:
              images = images.to(device)
              labels = labels.to(device)

              outputs = model(images)
              test_loss = criterion(outputs, labels)
              _, n_corrette=accuracy(outputs, labels)

              running_test_loss += test_loss.item()
              tot_corrette+=n_corrette.item()
              tot_eseguite+=labels.shape[0]

          test_acc=100* (tot_corrette/tot_eseguite)
          val_loss = running_test_loss / len(test_loader)
          acc_list_test.append(test_acc)
          print("Test acc: ", test_acc)
          print("Test loss: ", val_loss)


        if val_loss < best_loss:
          print("MIGLIORATO")
          torch.save(model.state_dict(), 'model_weights.pth')
          best_loss = val_loss
          best_model_train_acc=running_acc/seen
          best_model_test_acc=test_acc
          best_model_test_loss=val_loss
          best_model_train_loss=running_loss / len(train_loader)
          counter = 0
          # Salva i pesi del modello se la validation loss è migliorata
          torch.save(model.state_dict(), 'best_model.pt')
        else:
          counter += 1
        # Verifica se raggiunto il criterio di early stopping
          if counter >= patience:
              print(f'Early stopping at epoch {epoch+1}')
              stop=True
        print("BEST TEST LOSS: ", best_loss)

0


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch [0/100], Loss: 1.6768, Acc: 0.3486
Test acc:  47.47840624129284
Test loss:  1.4876196579595582
MIGLIORATO
BEST TEST LOSS:  1.4876196579595582
0
Epoch [1/100], Loss: 1.3834, Acc: 0.4922
Test acc:  52.57731958762887
Test loss:  1.302372357750361
MIGLIORATO
BEST TEST LOSS:  1.302372357750361
0
Epoch [2/100], Loss: 1.2931, Acc: 0.5165
Test acc:  54.527723599888546
Test loss:  1.247694233877469
MIGLIORATO
BEST TEST LOSS:  1.247694233877469
0
Epoch [3/100], Loss: 1.2597, Acc: 0.5295
Test acc:  55.48899414878796
Test loss:  1.2169924779275878
MIGLIORATO
BEST TEST LOSS:  1.2169924779275878
0
Epoch [4/100], Loss: 1.2372, Acc: 0.5391
Test acc:  56.60351072722207
Test loss:  1.1992898206267737
MIGLIORATO
BEST TEST LOSS:  1.1992898206267737
0
Epoch [5/100], Loss: 1.2151, Acc: 0.5482
Test acc:  56.882139871830596
Test loss:  1.1829513662156805
MIGLIORATO
BEST TEST LOSS:  1.1829513662156805
0
Epoch [6/100], Loss: 1.1975, Acc: 0.5540
Test acc:  56.770688213987185
Test loss:  1.1720840403463988


KeyboardInterrupt: ignored

In [ ]:
import zipfile


zip_ref = zipfile.ZipFile('/content/drive/Shareddrives/Datasets SEFAI/scraped_pictures.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, classification_report

model = vgg_face_dag(weights_path='model_weights.pth').to(device)


model.to(device)

# Definisci le trasformazioni per il test dataset e per scraped_pictures
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=mean, std=std)
])

# Calcola le metriche sul test dataset
model.eval()  # Imposta il modello in modalità di valutazione (non addestramento)
test_predictions = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(torch.nn.functional.softmax(outputs, dim=1), 1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions, average=None)
f1 = f1_score(test_labels, test_predictions, average=None)
#auc_roc = roc_auc_score(test_labels, test_predictions, multi_class='ovr')
classification_rep = classification_report(test_labels, test_predictions)

print("Test Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)
#print("AUC-ROC:", auc_roc)
print("Classification Report:\n", classification_rep)




/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Test Metrics:
Accuracy: 0.6613262747283366
Precision: [0.55278311 0.70588235 0.52       0.85541506 0.59153005 0.55767563
 0.76905312]
F1 Score: [0.576      0.33103448 0.45614035 0.86612858 0.64219503 0.53583333
 0.78491456]
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.60      0.58       958
           1       0.71      0.22      0.33       111
           2       0.52      0.41      0.46      1024
           3       0.86      0.88      0.87      1774
           4       0.59      0.70      0.64      1233
           5       0.56      0.52      0.54      1247
           6       0.77      0.80      0.78       831

    accuracy                           0.66      7178
   macro avg       0.65      0.59      0.60      7178
weighted avg       0.66      0.66      0.66      7178



/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


RuntimeError: ignored

In [ ]:
scraped_dataset = ImageFolder("photos", transform=test_transform)
scraped_loader = DataLoader(scraped_dataset, batch_size=64, shuffle=False)
# Calcola le metriche su scraped_pictures
scraped_predictions = []
scraped_labels = []

with torch.no_grad():
    for images, labels in scraped_loader:
        images = images.to(device)
        outputs = model(images)
        _, predictions = torch.max(torch.nn.functional.softmax(outputs, dim=1), 1)
        scraped_predictions.extend(predictions.cpu().numpy())
        scraped_labels.extend(labels)

accuracy_scraped = accuracy_score(scraped_labels, scraped_predictions)
precision_scraped = precision_score(scraped_labels, scraped_predictions, average=None)
f1_scraped = f1_score(scraped_labels, scraped_predictions, average=None)
#auc_roc_scraped = roc_auc_score(scraped_labels, scraped_predictions, multi_class='ovr')
classification_rep_scraped = classification_report(scraped_labels, scraped_predictions)

print("\nMetrics for scraped_pictures:")
print("Accuracy:", accuracy_scraped)
print("Precision:", precision_scraped)
print("F1 Score:", f1_scraped)
#print("AUC-ROC:", auc_roc_scraped)
print("Classification Report:\n", classification_rep_scraped)


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Metrics for scraped_pictures:
Accuracy: 0.31926121372031663
Precision: [0.46153846 0.         0.11111111 0.31506849 0.25       0.30188679
 0.66666667]
F1 Score: [0.47368421 0.         0.15217391 0.41071429 0.22222222 0.33684211
 0.29850746]
Classification Report:
               precision    recall  f1-score   support

           0       0.46      0.49      0.47        74
           1       0.00      0.00      0.00        74
           2       0.11      0.24      0.15        29
           3       0.32      0.59      0.41        78
           4       0.25      0.20      0.22        30
           5       0.30      0.38      0.34        42
           6       0.67      0.19      0.30        52

    accuracy                           0.32       379
   macro avg       0.30      0.30      0.27       379
weighted avg       0.31      0.32      0.28       379



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13